# Agent de Recherche d'Entreprise

Ce notebook implémente un outil interactif permettant de rechercher des informations sur des entreprises à partir de différentes sources en ligne. Il consolide automatiquement les données provenant de Wikipedia et Yahoo Finance dans une interface utilisateur conviviale.

## Fonctionnalités principales

- Recherche d'informations d'entreprise à partir de sources multiples
- Interface utilisateur interactive avec ipywidgets
- Exécution de requêtes parallèles pour optimiser les temps de recherche
- Historique des recherches avec sauvegarde automatique
- Exportation des résultats aux formats CSV et JSON
- Support multilingue (français, anglais, arabe)

## Architecture du code

Le code est organisé selon un pattern orienté objet avec les classes suivantes:
- `DataSource`: Classe abstraite pour les sources de données
- `WikipediaSource`: Récupère des descriptions depuis Wikipedia
- `YahooFinanceSource`: Récupère des données financières
- `CompanyResearchAgent`: Coordonne les recherches et gère l'historique
- `CompanyResearchUI`: Implémente l'interface utilisateur

## 1. Installation des dépendances

Commençons par installer les bibliothèques nécessaires:

In [ ]:
!pip install requests beautifulsoup4 wikipedia-api ipywidgets pandas yfinance

## 2. Importation des bibliothèques

Importons toutes les bibliothèques et modules nécessaires pour notre application:

In [ ]:
import requests
from bs4 import BeautifulSoup
import wikipediaapi
import json
import re
import pandas as pd
import yfinance as yf
from typing import Dict, Optional, List
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import logging
from concurrent.futures import ThreadPoolExecutor
import time
import os

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 3. Définition des classes pour les sources de données

Nous commençons par définir la classe abstraite `DataSource` et ses implémentations concrètes pour Wikipedia et Yahoo Finance:

In [ ]:
class DataSource:
    """Classe de base pour les sources de données

    Cette classe abstraite définit l'interface commune pour toutes les sources
    de données utilisées dans notre système de recherche d'entreprise.
    """
    def __init__(self, name: str):
        self.name = name

    def get_info(self, company_name: str) -> Dict:
        raise NotImplementedError

In [ ]:
class WikipediaSource(DataSource):
    """Source de données Wikipedia

    Cette classe permet de récupérer des informations générales sur une entreprise
    à partir de Wikipedia. Elle supporte plusieurs langues et retourne une description
    sommaire de l'entreprise ainsi que l'URL de la page correspondante.
    """
    def __init__(self, language: str = 'fr'):
        super().__init__("Wikipedia")
        self.wiki = wikipediaapi.Wikipedia(
            language=language,
            user_agent='CompanyResearchAgent/2.0 (contact@example.com)'
        )

    def get_info(self, company_name: str) -> Dict:
        try:
            page = self.wiki.page(company_name)
            if page.exists():
                return {
                    "description": self.clean_text(page.summary),
                    "url": page.fullurl,
                    "title": page.title
                }
        except Exception as e:
            logger.error(f"Erreur Wikipedia pour {company_name}: {e}")
        return None

    @staticmethod
    def clean_text(text: str) -> str:
        """Nettoie le texte en supprimant les références et les espaces superflus"""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\[.*?\]', '', text)
        return text.strip()

In [ ]:
class YahooFinanceSource(DataSource):
    """Source de données Yahoo Finance

    Cette classe permet de récupérer des informations financières sur une entreprise
    à partir de Yahoo Finance. Elle recherche d'abord le symbole boursier correspondant
    au nom de l'entreprise, puis récupère les informations détaillées.
    """
    def __init__(self):
        super().__init__("Yahoo Finance")

    def get_info(self, company_name: str) -> Dict:
        try:
            # Recherche du symbole boursier
            search_url = f"https://query2.finance.yahoo.com/v1/finance/search?q={company_name}"
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(search_url, headers=headers)
            search_data = response.json()

            if not search_data.get('quotes'):
                return None

            symbol = search_data['quotes'][0]['symbol']
            stock = yf.Ticker(symbol)
            info = stock.info

            return {
                "symbol": symbol,
                "sector": info.get('sector', 'N/A'),
                "industry": info.get('industry', 'N/A'),
                "market_cap": info.get('marketCap', 'N/A'),
                "employees": info.get('fullTimeEmployees', 'N/A'),
                "website": info.get('website', 'N/A')
            }
        except Exception as e:
            logger.error(f"Erreur Yahoo Finance pour {company_name}: {e}")
            return None

## 4. Définition de l'agent de recherche principal

Implémentons la classe `CompanyResearchAgent` qui coordonne les recherches et gère l'historique:

In [ ]:
class CompanyResearchAgent:
    """Agent principal de recherche d'entreprise

    Cette classe coordonne les recherches d'informations sur les entreprises à partir
    de plusieurs sources de données. Elle gère également l'historique des recherches,
    permettant de le sauvegarder et de le charger depuis un fichier JSON.
    """
    def __init__(self, language: str = 'fr'):
        self.sources = {
            'wikipedia': WikipediaSource(language),
            'yahoo': YahooFinanceSource()
        }
        self.search_history = []
        self.load_history()

    def load_history(self):
        """Charge l'historique des recherches depuis un fichier JSON"""
        if os.path.exists('search_history.json'):
            with open('search_history.json', 'r') as f:
                self.search_history = json.load(f)

    def save_history(self):
        """Sauvegarde l'historique des recherches dans un fichier JSON"""
        with open('search_history.json', 'w') as f:
            json.dump(self.search_history, f, indent=4)

    def research_company(self, company_name: str) -> Dict:
        """Effectue une recherche complète sur l'entreprise

        Utilise toutes les sources de données disponibles pour collecter
        des informations sur l'entreprise spécifiée. Les requêtes sont
        exécutées en parallèle pour optimiser le temps de recherche.

        Args:
            company_name: Nom de l'entreprise à rechercher

        Returns:
            Dictionnaire contenant toutes les informations collectées
        """
        start_time = time.time()
        results = {
            "nom_entreprise": company_name,
            "timestamp": datetime.now().isoformat(),
            "language": "fr",  # Ajout de l'attribut 'language'
            "success": True,   # Ajout de l'attribut 'success'
            "sources": {}
        }

        # Utilisation de ThreadPoolExecutor pour les requêtes parallèles
        with ThreadPoolExecutor(max_workers=len(self.sources)) as executor:
            future_to_source = {
                executor.submit(source.get_info, company_name): name
                for name, source in self.sources.items()
            }

            for future in future_to_source:
                source_name = future_to_source[future]
                try:
                    data = future.result()
                    if data:
                        results["sources"][source_name] = data
                except Exception as e:
                    logger.error(f"Erreur pour la source {source_name}: {e}")

        results["temps_execution"] = f"{time.time() - start_time:.2f} secondes"
        self.search_history.append(results)
        self.save_history()
        return results

## 5. Implémentation de l'interface utilisateur

Maintenant, créons la classe `CompanyResearchUI` qui fournit une interface graphique interactive pour notre agent de recherche:

In [ ]:
class CompanyResearchUI:
    """Interface utilisateur pour l'agent de recherche"""
    def __init__(self):
        self.agent = CompanyResearchAgent()
        self.setup_ui()

    def setup_ui(self):
        """Configure l'interface utilisateur avancée"""
        # Style CSS personnalisé
        display(HTML("""
        <style>
        .custom-button {
            margin: 10px 0;
            min-width: 120px;
        }
        .result-container {
            margin: 20px 0;
            padding: 15px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        </style>
        """))

        # Widgets d'entrée
        self.company_input = widgets.Text(
            description='Entreprise:',
            placeholder='Entrez le nom de l\'entreprise',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )

        self.language_select = widgets.Dropdown(
            options=[('Français', 'fr'), ('English', 'en'), ('Arabe', 'ar')],
            value='fr',
            description='Langue:',
            style={'description_width': 'initial'}
        )

        self.search_button = widgets.Button(
            description='Rechercher',
            button_style='info',
            icon='search',
            layout=widgets.Layout(width='200px')
        )

        self.clear_history_button = widgets.Button(
            description='Effacer historique',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='200px')
        )

        self.export_button = widgets.Button(
            description='Exporter',
            button_style='success',
            icon='download',
            layout=widgets.Layout(width='200px')
        )

        # Zone de sortie
        self.output = widgets.Output()
        self.history_output = widgets.Output()

        # Configuration des onglets
        self.tabs = widgets.Tab()
        self.tabs.children = [self.output, self.history_output]
        self.tabs.set_title(0, 'Résultats')
        self.tabs.set_title(1, 'Historique')

        # Événements
        self.search_button.on_click(self.on_search_click)
        self.clear_history_button.on_click(self.on_clear_history)
        self.export_button.on_click(self.on_export_click)

        # Layout principal
        main_container = widgets.VBox([
            widgets.HTML("<h2>🔍 Agent de Recherche d'Entreprise</h2>"),
            widgets.HBox([self.company_input, self.language_select]),
            widgets.HBox([self.search_button, self.clear_history_button, self.export_button]),
            self.tabs
        ])

        display(main_container)

    def format_results(self, results: Dict) -> str:
        """Formate les résultats pour l'affichage"""
        output = []
        output.append(f"🏢 Entreprise: {results['nom_entreprise']}")
        output.append(f"⏱️ Temps d'exécution: {results['temps_execution']}")
        output.append("\n")

        for source, data in results.get('sources', {}).items():
            output.append(f"📚 Source: {source}")
            for key, value in data.items():
                if isinstance(value, (int, float)) and value > 1000000:
                    value = f"{value:,}".replace(",", " ")
                output.append(f"• {key}: {value}")
            output.append("\n")

        return "\n".join(output)

    def on_search_click(self, b):
        """Gère le clic sur le bouton de recherche"""
        with self.output:
            clear_output()
            company_name = self.company_input.value.strip()

            if not company_name:
                print("⚠️ Veuillez entrer un nom d'entreprise.")
                return

            print(f"🔍 Recherche d'informations sur {company_name}...")
            self.agent = CompanyResearchAgent(self.language_select.value)
            results = self.agent.research_company(company_name)
            print("\n" + self.format_results(results))
            self.update_history()

    def update_history(self):
        """Met à jour l'affichage de l'historique avec plus de détails"""
        with self.history_output:
            clear_output()
            if not self.agent.search_history:
                display(HTML("<p>Aucune recherche dans l'historique.</p>"))
                return

            df = pd.DataFrame([{
                    'Entreprise': r['nom_entreprise'],
                    'Date': datetime.fromisoformat(r['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
                    'Langue': r['language'],
                    'Sources disponibles': ', '.join(r['sources'].keys()),
                    'Succès': '✅' if r['success'] else '❌',
                    'Temps': r['temps_execution']
                }
                for r in self.agent.search_history
            ])

            html_table = """
            <table style="width:100%; border-collapse: collapse; margin: 10px 0;">
                <thead>
                    <tr style="background-color: #f8f9fa;">
                        """ + ''.join('<th style="text-align:left; padding:10px; border:1px solid #ddd;">' + col + '</th>' for col in df.columns) + """
                    </tr>
                </thead>
                <tbody>
                    """ + ''.join(
                        "<tr>" + ''.join('<td style="padding:8px; border:1px solid #ddd;">' + str(cell) + '</td>' for cell in row) + "</tr>"
                        for _, row in df.iterrows()
                    ) + """
                </tbody>
            </table>
            """

            display(HTML(html_table))

    def on_clear_history(self, b):
        """Efface l'historique des recherches"""
        self.agent.search_history = []
        self.agent.save_history()
        self.update_history()

    def on_export_click(self, b):
        """Exporte l'historique des recherches"""
        if not self.agent.search_history:
            print("⚠️ Aucun historique à exporter.")
            return

        # Exportation en CSV
        df = pd.DataFrame([{
                'Entreprise': r['nom_entreprise'],
                'Date': datetime.fromisoformat(r['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
                'Langue': r['language'],
                'Sources disponibles': ', '.join(r['sources'].keys()),
                'Succès': '✅' if r['success'] else '❌',
                'Temps': r['temps_execution']
            }
            for r in self.agent.search_history
        ])

        df.to_csv('historique_recherches.csv', index=False)
        with open('historique_recherches.json', 'w') as f:
            json.dump(self.agent.search_history, f, indent=4)

        print("✅ Historique exporté avec succès (CSV et JSON).")


In [ ]:
    def format_results(self, results: Dict) -> str:
        """Formate les résultats pour l'affichage

        Convertit le dictionnaire de résultats en une chaîne de texte formatée
        pour une meilleure lisibilité dans l'interface utilisateur.
        """
        output = []
        output.append(f"🏢 Entreprise: {results['nom_entreprise']}")
        output.append(f"⏱️ Temps d'exécution: {results['temps_execution']}")
        output.append("\n")

        for source, data in results.get('sources', {}).items():
            output.append(f"📚 Source: {source}")
            for key, value in data.items():
                if isinstance(value, (int, float)) and value > 1000000:
                    value = f"{value:,}".replace(",", " ")
                output.append(f"• {key}: {value}")
            output.append("\n")

        return "\n".join(output)

    def on_search_click(self, b):
        """Gère le clic sur le bouton de recherche

        Récupère le nom de l'entreprise, lance la recherche et affiche les résultats.
        """
        with self.output:
            clear_output()
            company_name = self.company_input.value.strip()

            if not company_name:
                print("⚠️ Veuillez entrer un nom d'entreprise.")
                return

            print(f"🔍 Recherche d'informations sur {company_name}...")
            self.agent = CompanyResearchAgent(self.language_select.value)
            results = self.agent.research_company(company_name)
            print("\n" + self.format_results(results))
            self.update_history()

In [ ]:
    def update_history(self):
        """Met à jour l'affichage de l'historique avec plus de détails

        Crée un tableau HTML contenant l'historique des recherches effectuées.
        """
        with self.history_output:
            clear_output()
            if not self.agent.search_history:
                display(HTML("<p>Aucune recherche dans l'historique.</p>"))
                return

            df = pd.DataFrame([{
                    'Entreprise': r['nom_entreprise'],
                    'Date': datetime.fromisoformat(r['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
                    'Langue': r['language'],
                    'Sources disponibles': ', '.join(r['sources'].keys()),
                    'Succès': '✅' if r['success'] else '❌',
                    'Temps': r['temps_execution']
                }
                for r in self.agent.search_history
            ])

            html_table = """
            <table style="width:100%; border-collapse: collapse; margin: 10px 0;">
                <thead>
                    <tr style="background-color: #f8f9fa;">
                        """ + ''.join('<th style="text-align:left; padding:10px; border:1px solid #ddd;">' + col + '</th>' for col in df.columns) + """
                    </tr>
                </thead>
                <tbody>
                    """ + ''.join(
                        "<tr>" + ''.join('<td style="padding:8px; border:1px solid #ddd;">' + str(cell) + '</td>' for cell in row) + "</tr>"
                        for _, row in df.iterrows()
                    ) + """
                </tbody>
            </table>
            """

            display(HTML(html_table))

    def on_clear_history(self, b):
        """Efface l'historique des recherches"""
        self.agent.search_history = []
        self.agent.save_history()
        self.update_history()

    def on_export_click(self, b):
        """Exporte l'historique des recherches

        Enregistre l'historique des recherches aux formats CSV et JSON.
        """
        if not self.agent.search_history:
            print("⚠️ Aucun historique à exporter.")
            return

        # Exportation en CSV
        df = pd.DataFrame([{
                'Entreprise': r['nom_entreprise'],
                'Date': datetime.fromisoformat(r['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
                'Langue': r['language'],
                'Sources disponibles': ', '.join(r['sources'].keys()),
                'Succès': '✅' if r['success'] else '❌',
                'Temps': r['temps_execution']
            }
            for r in self.agent.search_history
        ])

        df.to_csv('historique_recherches.csv', index=False)
        with open('historique_recherches.json', 'w') as f:
            json.dump(self.agent.search_history, f, indent=4)

        print("✅ Historique exporté avec succès (CSV et JSON).")

## 6. Fonction principale et lancement de l'application

Enfin, définissons la fonction principale et lançons l'application:

In [ ]:
def main():
    """Fonction principale

    Crée et initialise l'interface utilisateur de recherche d'entreprise.
    """
    ui = CompanyResearchUI()
    return ui  # Retourne l'objet UI pour y accéder ultérieurement si nécessaire

# Lancer l'application si ce notebook est exécuté directement
if __name__ == "__main__":
    app = main()

✅ Historique exporté avec succès (CSV et JSON).


## 7. Exemple d'utilisation

Voici comment vous pouvez utiliser cette application:

1. Exécutez toutes les cellules de ce notebook
2. L'interface utilisateur s'affichera automatiquement
3. Entrez le nom d'une entreprise dans le champ "Entreprise"
4. Sélectionnez la langue souhaitée
5. Cliquez sur le bouton "Rechercher"
6. Les résultats s'afficheront dans l'onglet "Résultats"
7. L'historique des recherches sera visible dans l'onglet "Historique"
8. Vous pouvez exporter l'historique au format CSV et JSON en cliquant sur "Exporter"

Exemple de recherche: entrez "Apple" ou "Total" ou "Airbus" pour tester l'application.